In [1]:
from pynq.overlays.base import BaseOverlay
import time
import threading
import pynq.lib.rgbled as rgbled
from datetime import datetime
base = BaseOverlay("base.bit")

## Part A2.2

In [2]:
import asyncio
import random
flag = 1 #flag used to run code forever
cond = True #condition check for button interrupt

# function to execute philosopher's actions (represented by blinks)
def phil_action(t,d,n):
    '''
    Function to blink the LEDs depending on Philosopher's state
    Parameters:
      t: number of times to blink the LED
      d: duration (in seconds) to blink
      n: index of Philosopher LED (0 to 4)
    '''
    led4 = rgbled.RGBLED(4) #define LED for Philosopher4
    mask = 0x6              #set mask to toggle value for blinking LED
    if n < 4 :
        if t == 0: #if action is to not blink, keep LED off
            base.leds[n].off()
        else:
            for i in range(t):
                base.leds[n].toggle()   # toggle LED light to blink
                time.sleep(d)           # duration for blink
    elif n == 4:
        if t ==0: #if action is to not blink, keep LED off
            led4.write(0)
        else:
            for i in range(t):
                led4.write(mask)      #set value of LED depending on Mask ( will be 1 or 0, to blink)
                time.sleep(d)        # duration to blink
                mask = mask ^ 0x6   # do XOR in order to toggle LED

def resetLeds(): #function to clear all LEDs
    base.leds[0].off()
    base.leds[1].off()
    base.leds[2].off()
    base.leds[3].off()
    led4 = rgbled.RGBLED(4)
    led4.write(0)
        
def worker_t(i,forks):
    '''
    Worker function to try and acquire 2 forks and blink the LED to eat or sleep.  If no forks available, no blink
    forks: 2 threading locks (resource)
    i: index representing the Philosopher/LED number.
    '''
    fork1Index = i; #set Philospher's fork1 value
    fork2Index = (i+1)%5; #calculate Philosopher's fork2 value
    eatTime = random.randint(40,80) #set eating time
    napTime = random.randint(10,20) #set napping time
    state = "Starving";  # set initial state of state machine
    
    while(flag):
        if state == "Starving": #if in Starving state, check for Philospher's 2 forks
            fork1_available = forks[fork1Index].acquire(False)
            fork2_available = forks[fork2Index].acquire(False)
            print('Philosopher{} Checked for fork{} and fork{}'.format(i,fork1Index,fork2Index))
            if fork1_available and fork2_available: #if 2 forks available, change to Eating state
                state = "Eating"
            else:
                if fork1_available == False: #if fork1 was not available, print message
                    print('fork{} was not available'.format(fork1Index))
                else:
                    forks[fork1Index].release() #if fork1 was available, but not fork2, release fork1
                if fork2_available == False: #if fork2 was not available, print message
                    print('fork{} was not available'.format(fork2Index))
                else:
                    forks[fork2Index].release() #if fork2 was available, but not fork1, release fork2
                print('Philosopher{} is Starving! waiting for forks'.format(i))
                phil_action(0,0,i); #LED will not blink, and Philosopher waits
                time.sleep(10)
        elif state == "Eating": #if Eating state, blink Philosopher
                print('Philosopher{} is eating!'.format(i))
                phil_action(eatTime,.1,i) #LED will blinks fast as Philosopher eats
                forks[fork1Index].release()
                print('released fork{} by Philosopher{}!'.format(fork1Index,i))
                forks[fork2Index].release()
                print('released fork{} by Philosopher{}!'.format(fork2Index,i))
                state = "Napping";    
    
        elif state == "Napping": #if Napping state, blink Philosopher
            print('Philosopher{} naps'.format(i))
            phil_action(napTime,.5,i) #LED will blink slow as Philosopher naps
            state = "Starving"


In [3]:
# function to asynchronously check button interrupt
async def get_btns(_loop):
    global flag, cond, start
    while cond: #run loop while cond is True
        await asyncio.sleep(0.01)
        if base.buttons[0].read() != 0: #if button 0 was pressed, set flag to 0
            flag = 0
            print('Program Interrupted by Button{}'.format(0))
            resetLeds()
            time.sleep(5)
            cond = False #set condition to False, since button was pressed
        elif base.buttons[1].read() != 0: #if button 1 was pressed, set flag to 0
            flag = 0
            print('Program Interrupted by Button{}'.format(1))
            resetLeds()
            time.sleep(5)
            cond = False #set condition to False, since button was pressed
        elif base.buttons[2].read() != 0: #if button 2 was pressed, set flag to 0
            flag = 0
            print('Program Interrupted by Button{}'.format(2))
            resetLeds()
            time.sleep(5)
            cond = False #set condition to False, since button was pressed
        elif base.buttons[3].read() != 0: #if button 3 was pressed, set flag to 0
            flag = 0
            print('Program Interrupted by Button{}'.format(3))
            resetLeds()
            time.sleep(5)
            cond = False #set condition to False, since button was pressed

    _loop.stop() #end forever loop because of button interrupt

#define array of forks (i.e locks), 5 for the 5 Philosophers to share
forks = []
for f in range(5):
    forks.append(threading.Lock())
    
#define array of threads, each representing the Philosophers in Eating, Napping and Starving phase
threads = []
for i in range(5):
    t = threading.Thread(target=worker_t, args=(i, forks))
    threads.append(t)
    t.start()

loop = asyncio.new_event_loop() #define loop
loop.create_task(get_btns(loop)) # create button reader task to loop
loop.run_forever() #run loop infinitely

for t in threads:
    name = t.getName()
    t.join()
    print('{} joined'.format(name))


Philosopher0 Checked for fork0 and fork1
Philosopher0 is eating!
Philosopher1 Checked for fork1 and fork2Philosopher2 Checked for fork2 and fork3
fork1 was not available
Philosopher1 is Starving! waiting for forks

fork2 was not available
Philosopher2 is Starving! waiting for forks
Philosopher3 Checked for fork3 and fork4Philosopher4 Checked for fork4 and fork0
Philosopher3 is eating!

fork4 was not available
fork0 was not available
Philosopher4 is Starving! waiting for forks
released fork3 by Philosopher3!
released fork4 by Philosopher3!
Philosopher3 naps
released fork0 by Philosopher0!
released fork1 by Philosopher0!
Philosopher0 naps
Philosopher1 Checked for fork1 and fork2
Philosopher1 is eating!
Philosopher2 Checked for fork2 and fork3
fork2 was not available
Philosopher2 is Starving! waiting for forks
Philosopher4 Checked for fork4 and fork0
Philosopher4 is eating!
Philosopher0 Checked for fork0 and fork1
fork0 was not available
fork1 was not available
Philosopher0 is Starving! w

### Conclusion for A2.2:
After implementing the system in A2.1, the next system design update was to randomize the eating and napping time.  To do this, I included the random library to use function random.randint.  This function generates random values between int value a, b .  A design consideration for this was to add boundaries to the random eating/napping times, such that eating time and napping time did not cause a deadlock in the starving state.  For this I chose values between 40 and 80 for eating time and  10 and 20 for napping time.